# Cryptocurrency Model building

*Mohammed Jamil*

## Context

The first half of 2022 has been very bad for the crypto market. Crypto industry experts have mixed opinions about the future of cryptocurrencies. While some believe the market will continue to be volatile, others expect some stability in the second half of 2022.

In these uncertain times for the crypto market, is it still worth the risk investing in cryptocurrencies ? and what are the low-risk cryptocurrencies to consider ?

## Table of Contents :

- Importing data

## Dependencies :

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense, LSTM

2022-09-21 19:49:22.646632: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-09-21 19:49:27.214751: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2022-09-21 19:49:28.748358: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-09-21 19:49:28.748392: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if yo

---

### Importing Data :

In the previous notebook (Cryto market analysis) we took a look at the tendencies of the crypto market to find out what are the top 10 cryptocurrencies currently, and exported the historical daily prices data into a csv file. Firstly, Let's start by loading the data.

NB: If you are not using bash juste comment the next cell and run the previous notebook.

In [16]:
%%bash

CSV_FILE=./top10-crypto-daily-prices.csv

if [ ! -f "$CSV_FILE" ]; then
    python3 crypto-daily-prices.py
fi

In [20]:
# Reading data from csv.

file = "top10-crypto-daily-prices.csv"
daily_prices_df = pd.read_csv(file, index_col=0)
print("DataFrame shape :",daily_prices_df.shape)
daily_prices_df.head()

DataFrame shape : (22387, 3)


,id,date,price
0,bitcoin,2013-04-28,135.30
1,bitcoin,2013-04-29,141.96
2,bitcoin,2013-04-30,135.30
3,bitcoin,2013-05-01,117.00
4,bitcoin,2013-05-02,103.43
